In [14]:
import os
import json
from datetime import datetime
import redis

In [9]:
YOUTUBE_METADATA_DIR = "/Volumes/media/youtube/metadata"
YOUTUBE_THUMBNAIL_DIR = "/Volumes/media/youtube/thumbnails"
YOUTUBE_VIDEO_DIR = "/Volumes/media/youtube/videos"

## Discover Videos

In [2]:
files = os.listdir(YOUTUBE_VIDEO_DIR)
print(f'found {len(files)} files in {YOUTUBE_VIDEO_DIR}')
with open('data/files.json', 'w') as outfile:
    json.dump(files, outfile)

## Dedupe Videos

In [38]:
filename_idx = {}

with open('data/file_list_videos.json') as json_file:
    data = json.load(json_file)
    print(len(data))
    for file in data:
        filename, ext = os.path.splitext(file)
        if filename in filename_idx:
            filename_idx[filename].append(ext)
        else:
            filename_idx[filename] = [ext]

dupes = []
for filename, extensions in filename_idx.items():
    if len(extensions) > 1:
        dupes.append(filename)

remove = []
print(f'found {len(dupes)} duplicate filenames')
for dup in dupes:
    sizes = {}
    for ext in filename_idx[dup]:
        size = os.path.getsize(os.path.join(YOUTUBE_VIDEO_DIR, f'{dup}{ext}'))
        sizes[ext] = size

    # if len(sizes) > 2:
    #     print(f'{dup} has {len(sizes)} copies: {sizes}')

    smallest = min(sizes, key=sizes.get)

    # print(f'Keeping: {dup}{smallest} @ {sizes[smallest]}')
    sizes.pop(smallest)
    for ext in sizes:
        # print(f'Removing: {dup}{ext} @ {sizes[ext]}')
        remove.append(f'{dup}{ext}')

print(f'found {len(remove)} files to remove')

for file in remove:
    os.remove(os.path.join(YOUTUBE_VIDEO_DIR, file))
    data.remove(file)

print(len(data))

# write the json file
with open('data/file_list_videos_deduped.json', 'w') as outfile:
    json.dump(data, outfile)

29893
found 45 duplicate filenames
found 50 files to remove
29843


## Missing Ids

In [50]:
fragments = set()

with open('data/file_list_videos_deduped.json') as json_file:
    data = json.load(json_file)

    for file in data:
        filename, ext = os.path.splitext(file)

        # extract the youtube id from the filename
        id = filename[-12:]

        # handle fragments
        if '.' in id:
            filename, ext = os.path.splitext(filename)
            id = filename[-11:]
            fragments.add(id)

        elif id[0] != '-':
            print(f'{id}|{filename}')

print(f'found {len(fragments)} fragments')
print(fragments)

# write the json file
with open('data/fragments.json', 'w') as outfile:
    json.dump(list(fragments), outfile)

found 6 fragments
{'UPIVxo6fo3M', 'eZ1rpiEy6Ic', 'd7OxfJZOIhQ', 'dgG10nN6oDM', 'zWxSZcpeS8Q', 'b8cFILGrUo0'}


In [39]:
len("-y7DXX6_RBz8")
# # rename file
# os.rename(os.path.join(YOUTUBE_VIDEO_DIR, old_file), os.path.join(YOUTUBE_VIDEO_DIR, new_file))

12

In [ ]:
    # # Get channel details
    # request = youtube.channels().list(
    #     part="snippet,contentDetails,statistics",
    #     mine=True
    #     # id="UCqsUJL5xIWuidR7sIrPLhAw",
    # )
    # request = youtube.search().list(
    #     channelId = CHANNEL_ID,
    #     part = 'id,snippet',
    #     type = 'video',
    #     publishedAfter = '2018-12-31T23:59:59Z',
    #     publishedBefore = '2020-01-01T00:00:00Z',
    #     order = 'date',
    #     fields = 'nextPageToken,items(id,snippet)',
    #     maxResults = 50
    # )
    # video_data = {}
